In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully")

## 1. Load Data

In [ ]:
# Load cleaned panel data
panel_data = pd.read_csv('../data/cleaned_panel_data.csv')

# Load ML risk scores
risk_scores = pd.read_csv('../data/country_risk_scores.csv')

# Load focus countries
with open('../data/focus_countries.txt', 'r') as f:
    focus_countries = f.read().strip().split(',')

print(f"Panel data shape: {panel_data.shape}")
print(f"Risk scores shape: {risk_scores.shape}")
print(f"Focus countries: {', '.join(focus_countries)}")
print(f"\nAvailable indicators: {panel_data.columns.tolist()[3:]}")

## 2. Debt Sustainability Indicators

In [ ]:
# Merge panel data with risk scores
fiscal_data = panel_data.merge(
    risk_scores[['country', 'year', 'risk_score']], 
    left_on=['country', 'year'], 
    right_on=['country', 'year'],
    how='left'
).drop(columns=['country', 'year'])

# Calculate debt-to-GDP ratio (if not already present)
if 'debt_to_gdp' not in fiscal_data.columns:
    fiscal_data['debt_to_gdp'] = (fiscal_data['government_debt'] / fiscal_data['nominal_gdp']) * 100

# Calculate deficit-to-GDP
fiscal_data['budget_balance'] = fiscal_data['revenue'] - fiscal_data['expenditure']
fiscal_data['deficit_to_gdp'] = (fiscal_data['budget_balance'] / fiscal_data['nominal_gdp']) * 100

# Calculate debt service metrics (if interest payments available)
if 'interest_payments' in fiscal_data.columns:
    fiscal_data['debt_service_to_revenue'] = (fiscal_data['interest_payments'] / fiscal_data['revenue']) * 100
    fiscal_data['debt_service_to_gdp'] = (fiscal_data['interest_payments'] / fiscal_data['nominal_gdp']) * 100
    
print("✅ Debt sustainability indicators calculated")
print(f"\nNew metrics created:")
print(f"  - debt_to_gdp")
print(f"  - deficit_to_gdp")
print(f"  - budget_balance")
if 'debt_service_to_revenue' in fiscal_data.columns:
    print(f"  - debt_service_to_revenue")
    print(f"  - debt_service_to_gdp")

### 2.1 Debt-to-GDP Trends

In [ ]:
# Focus on recent years and focus countries
recent_years = fiscal_data['year'].max() - 20
focus_fiscal = fiscal_data[
    (fiscal_data['country'].isin(focus_countries)) & 
    (fiscal_data['year'] >= recent_years)
]

# Plot debt-to-GDP evolution
fig = px.line(
    focus_fiscal, 
    x='year', 
    y='debt_to_gdp', 
    color='country',
    title='Debt-to-GDP Ratio Evolution (Focus Countries)',
    labels={'debt_to_gdp': 'Debt-to-GDP Ratio (%)', 'year': 'year'},
    height=600
)

# Add sustainability threshold lines
fig.add_hline(y=70, line_dash="dash", line_color="red", 
              annotation_text="High Risk Threshold (70%)")
fig.add_hline(y=50, line_dash="dash", line_color="orange", 
              annotation_text="Moderate Risk (50%)")
fig.add_hline(y=30, line_dash="dash", line_color="green", 
              annotation_text="Low Risk (30%)")

fig.update_layout(hovermode='x unified')
fig.show()

print("\n📊 Debt-to-GDP Sustainability Assessment:")
print("   < 30%: Low risk (sustainable)")
print("   30-50%: Moderate risk (monitor closely)")
print("   50-70%: Elevated risk (policy action needed)")
print("   > 70%: High risk (urgent intervention required)")

In [ ]:
# Current debt levels by country
latest_year = fiscal_data['year'].max()
latest_debt = fiscal_data[
    fiscal_data['year'] == latest_year
][['country', 'debt_to_gdp', 'risk_score']].sort_values('debt_to_gdp', ascending=False)

print(f"\n📊 Debt-to-GDP Levels ({latest_year}):")
print("="*70)
for _, row in latest_debt.iterrows():
    debt = row['debt_to_gdp']
    risk = row['risk_score']
    status = "🔴" if debt > 70 else "🟡" if debt > 50 else "🟢"
    if pd.notna(debt):
        risk_str = f"{risk:.1f}%" if pd.notna(risk) else "N/A"
        print(f"{status} {row['country']:20s}: {debt:5.1f}% debt  |  {risk_str:>6s} ML risk")

### 2.2 Fiscal Deficit Analysis

In [ ]:
# Plot deficit trends
fig = px.line(
    focus_fiscal, 
    x='year', 
    y='deficit_to_gdp', 
    color='country',
    title='Budget Deficit/Surplus as % of GDP (Focus Countries)',
    labels={'deficit_to_gdp': 'Budget Balance (% of GDP)', 'year': 'year'},
    height=600
)

# Add reference lines
fig.add_hline(y=0, line_dash="solid", line_color="black", 
              annotation_text="Balanced Budget")
fig.add_hline(y=-3, line_dash="dash", line_color="orange", 
              annotation_text="-3% (Moderate deficit)")
fig.add_hline(y=-5, line_dash="dash", line_color="red", 
              annotation_text="-5% (High deficit)")

fig.update_layout(hovermode='x unified')
fig.show()

print("\n📊 Fiscal Balance Interpretation:")
print("   > 0%: Surplus (fiscal space available)")
print("   0 to -3%: Moderate deficit (manageable)")
print("   -3% to -5%: Elevated deficit (concerning)")
print("   < -5%: Large deficit (unsustainable)")

## 3. Revenue Efficiency Analysis

In [ ]:
# Calculate revenue metrics
fiscal_data['revenue_to_gdp'] = (fiscal_data['revenue'] / fiscal_data['nominal_gdp']) * 100
fiscal_data['tax_revenue_to_gdp'] = (fiscal_data['tax_revenue'] / fiscal_data['nominal_gdp']) * 100
fiscal_data['tax_effort'] = (fiscal_data['tax_revenue'] / fiscal_data['revenue']) * 100

print("✅ Revenue metrics calculated")

# Recent revenue performance
recent_revenue = fiscal_data[
    (fiscal_data['year'] >= latest_year - 5) & 
    (fiscal_data['country'].isin(focus_countries))
].groupby('country').agg({
    'revenue_to_gdp': 'mean',
    'tax_revenue_to_gdp': 'mean',
    'tax_effort': 'mean'
}).round(2).sort_values('revenue_to_gdp', ascending=False)

print("\n📊 Revenue Performance (Last 5 Years Average):")
print("="*70)
print(recent_revenue)

print("\n💡 Benchmarks:")
print("   Revenue-to-GDP: 15-25% typical for developing countries")
print("   Tax Revenue-to-GDP: >15% indicates strong tax capacity")
print("   Tax Effort: >60% shows high reliance on tax revenue")

In [ ]:
# Revenue efficiency comparison
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Revenue-to-GDP Ratio', 'Tax Effort (Tax/Total Revenue)'),
    specs=[[{"type": "bar"}, {"type": "bar"}]]
)

# Revenue-to-GDP
fig.add_trace(
    go.Bar(
        x=recent_revenue.index,
        y=recent_revenue['revenue_to_gdp'],
        marker_color='steelblue',
        showlegend=False
    ),
    row=1, col=1
)

# Tax effort
fig.add_trace(
    go.Bar(
        x=recent_revenue.index,
        y=recent_revenue['tax_effort'],
        marker_color='coral',
        showlegend=False
    ),
    row=1, col=2
)

fig.update_yaxes(title_text="% of GDP", row=1, col=1)
fig.update_yaxes(title_text="% of Revenue", row=1, col=2)
fig.update_xaxes(tickangle=-45)
fig.update_layout(height=500, title_text="Revenue Efficiency Metrics")
fig.show()

## 4. Expenditure Composition Analysis

In [ ]:
# Calculate expenditure metrics
fiscal_data['expenditure_to_gdp'] = (fiscal_data['expenditure'] / fiscal_data['nominal_gdp']) * 100

# Calculate social spending if available
if 'health_expenditure' in fiscal_data.columns and 'education_expenditure' in fiscal_data.columns:
    fiscal_data['social_spending'] = fiscal_data['health_expenditure'] + fiscal_data['education_expenditure']
    fiscal_data['social_to_gdp'] = (fiscal_data['social_spending'] / fiscal_data['nominal_gdp']) * 100
    fiscal_data['social_share'] = (fiscal_data['social_spending'] / fiscal_data['expenditure']) * 100

# Interest payments as share of expenditure
if 'interest_payments' in fiscal_data.columns:
    fiscal_data['interest_share'] = (fiscal_data['interest_payments'] / fiscal_data['expenditure']) * 100

print("✅ Expenditure composition metrics calculated")

In [ ]:
# Recent expenditure patterns
exp_cols = ['expenditure_to_gdp']
if 'social_to_gdp' in fiscal_data.columns:
    exp_cols.append('social_to_gdp')
if 'debt_service_to_gdp' in fiscal_data.columns:
    exp_cols.append('debt_service_to_gdp')

recent_expenditure = fiscal_data[
    (fiscal_data['year'] >= latest_year - 5) & 
    (fiscal_data['country'].isin(focus_countries))
].groupby('country')[exp_cols].mean().round(2).sort_values('expenditure_to_gdp', ascending=False)

print("\n📊 Expenditure Composition (Last 5 Years Average):")
print("="*70)
print(recent_expenditure)

print("\n💡 Analysis:")
print("   High expenditure-to-GDP with low social spending = potential misallocation")
print("   High debt service crowds out productive investments")

In [ ]:
# Expenditure-Revenue balance
recent_balance = fiscal_data[
    (fiscal_data['year'] >= latest_year - 5) & 
    (fiscal_data['country'].isin(focus_countries))
].groupby('country').agg({
    'revenue_to_gdp': 'mean',
    'expenditure_to_gdp': 'mean',
    'deficit_to_gdp': 'mean'
}).round(2)

# Create comparison chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=recent_balance.index,
    y=recent_balance['revenue_to_gdp'],
    name='revenue',
    marker_color='green'
))

fig.add_trace(go.Bar(
    x=recent_balance.index,
    y=recent_balance['expenditure_to_gdp'],
    name='expenditure',
    marker_color='red'
))

fig.update_layout(
    title='Revenue vs Expenditure (% of GDP) - Last 5 Years Average',
    xaxis_title='country',
    yaxis_title='% of GDP',
    height=600,
    barmode='group',
    xaxis={'tickangle': -45}
)
fig.show()

print("\n💡 Insight: Gap between bars indicates fiscal deficit magnitude")

## 5. Integrated Fiscal Health Assessment

In [ ]:
# Create comprehensive fiscal health scorecard
scorecard_data = fiscal_data[
    (fiscal_data['year'] >= latest_year - 3) & 
    (fiscal_data['country'].isin(focus_countries))
].groupby('country').agg({
    'debt_to_gdp': 'mean',
    'deficit_to_gdp': 'mean',
    'revenue_to_gdp': 'mean',
    'expenditure_to_gdp': 'mean',
    'risk_score': 'mean'
}).round(2)

# Add fiscal health score (0-100, lower is better)
def calculate_fiscal_health(row):
    """
    Calculate fiscal health score based on multiple indicators.
    Lower score = better fiscal health
    """
    score = 0
    
    # Debt component (0-40 points)
    if pd.notna(row['debt_to_gdp']):
        score += min(40, row['debt_to_gdp'] / 2)  # 80% debt = 40 points
    
    # Deficit component (0-30 points)
    if pd.notna(row['deficit_to_gdp']):
        score += min(30, abs(row['deficit_to_gdp']) * 3)  # -10% deficit = 30 points
    
    # Revenue component (0-15 points, inverse)
    if pd.notna(row['revenue_to_gdp']):
        score += max(0, 15 - row['revenue_to_gdp'])  # <15% revenue = 15 points
    
    # ML risk component (0-15 points)
    if pd.notna(row['risk_score']):
        score += row['risk_score'] * 0.15  # 100% risk = 15 points
    
    return min(100, score)

scorecard_data['fiscal_stress_score'] = scorecard_data.apply(calculate_fiscal_health, axis=1).round(1)
scorecard_data = scorecard_data.sort_values('fiscal_stress_score', ascending=False)

print("="*70)
print("COMPREHENSIVE FISCAL HEALTH SCORECARD (Last 3 Years)")
print("="*70)
print(scorecard_data)
print("\n💡 Fiscal Stress Score: 0=Excellent, 100=Severe stress")
print("   Components: Debt (40%), Deficit (30%), Revenue (15%), ML Risk (15%)")

In [ ]:
# Visualize fiscal health scorecard
fig = go.Figure()

# Add bars for fiscal stress score
colors = ['darkred' if x > 70 else 'orange' if x > 40 else 'yellow' if x > 20 else 'green' 
          for x in scorecard_data['fiscal_stress_score']]

fig.add_trace(go.Bar(
    x=scorecard_data.index,
    y=scorecard_data['fiscal_stress_score'],
    marker_color=colors,
    text=scorecard_data['fiscal_stress_score'],
    textposition='outside'
))

fig.update_layout(
    title='Fiscal Stress Score by Country (0=Best, 100=Worst)',
    xaxis_title='country',
    yaxis_title='Fiscal Stress Score',
    height=600,
    xaxis={'tickangle': -45},
    showlegend=False
)

# Add threshold lines
fig.add_hline(y=70, line_dash="dash", line_color="red", annotation_text="Severe")
fig.add_hline(y=40, line_dash="dash", line_color="orange", annotation_text="Moderate")
fig.add_hline(y=20, line_dash="dash", line_color="yellow", annotation_text="Low")

fig.show()

## 6. Debt Sustainability Matrix

In [ ]:
# Create debt-deficit sustainability matrix
latest_matrix = fiscal_data[
    (fiscal_data['year'] >= latest_year - 3)
].groupby('country').agg({
    'debt_to_gdp': 'mean',
    'deficit_to_gdp': 'mean',
    'risk_score': 'mean'
}).dropna()

# Create scatter plot
fig = px.scatter(
    latest_matrix.reset_index(),
    x='debt_to_gdp',
    y='deficit_to_gdp',
    size='risk_score',
    color='risk_score',
    text='country',
    title='Debt Sustainability Matrix: Debt vs Deficit',
    labels={
        'debt_to_gdp': 'Debt-to-GDP Ratio (%)',
        'deficit_to_gdp': 'Budget Balance (% of GDP)',
        'risk_score': 'ML Risk Score'
    },
    color_continuous_scale='RdYlGn_r',
    height=700
)

# Add quadrant lines
fig.add_vline(x=70, line_dash="dash", line_color="red")
fig.add_hline(y=-5, line_dash="dash", line_color="red")
fig.add_hline(y=0, line_dash="solid", line_color="black")

# Add quadrant annotations
fig.add_annotation(x=85, y=2, text="High Debt<br>Surplus", showarrow=False)
fig.add_annotation(x=85, y=-7, text="High Debt<br>High Deficit<br>⚠️ CRISIS ZONE", 
                  showarrow=False, font=dict(color='red', size=12))
fig.add_annotation(x=40, y=2, text="Low Debt<br>Surplus<br>✅ SUSTAINABLE", 
                  showarrow=False, font=dict(color='green', size=12))
fig.add_annotation(x=40, y=-7, text="Low Debt<br>Deficit", showarrow=False)

fig.update_traces(textposition='top center')
fig.update_layout(showlegend=True)
fig.show()

print("\n💡 Matrix Interpretation:")
print("   Top-Left: High debt + Surplus = Deleveraging (improving)")
print("   Top-Right: High debt + High deficit = CRISIS (urgent action)")
print("   Bottom-Left: Low debt + Surplus = Sustainable (ideal)")
print("   Bottom-Right: Low debt + Deficit = Manageable (monitor)")

## 7. Export Fiscal Metrics

In [ ]:
# Save comprehensive fiscal metrics
fiscal_data.to_csv('../data/fiscal_metrics_complete.csv', index=False)
print("✅ Saved: fiscal_metrics_complete.csv")

# Save fiscal health scorecard
scorecard_data.to_csv('../data/fiscal_health_scorecard.csv')
print("✅ Saved: fiscal_health_scorecard.csv")

# Save sustainability matrix
latest_matrix.to_csv('../data/debt_sustainability_matrix.csv')
print("✅ Saved: debt_sustainability_matrix.csv")

# Export focus country metrics for dashboard
focus_metrics = fiscal_data[
    fiscal_data['country'].isin(focus_countries)
][['country', 'year', 'debt_to_gdp', 'deficit_to_gdp', 'revenue_to_gdp', 
   'expenditure_to_gdp', 'risk_score']].dropna()
focus_metrics.to_csv('../data/focus_country_fiscal_metrics.csv', index=False)
print("✅ Saved: focus_country_fiscal_metrics.csv")

## 8. Key Policy Recommendations

In [ ]:
print("="*70)
print("FISCAL SUSTAINABILITY - KEY FINDINGS & RECOMMENDATIONS")
print("="*70)

# Identify high-stress countries
high_stress = scorecard_data[scorecard_data['fiscal_stress_score'] > 50].index.tolist()
moderate_stress = scorecard_data[
    (scorecard_data['fiscal_stress_score'] > 30) & 
    (scorecard_data['fiscal_stress_score'] <= 50)
].index.tolist()

print("\n1️⃣ COUNTRIES REQUIRING URGENT INTERVENTION:")
for country in high_stress:
    data = scorecard_data.loc[country]
    print(f"   🔴 {country}:")
    print(f"      - Fiscal Stress Score: {data['fiscal_stress_score']:.1f}/100")
    print(f"      - Debt-to-GDP: {data['debt_to_gdp']:.1f}%")
    print(f"      - Budget Deficit: {data['deficit_to_gdp']:.1f}% of GDP")
    print(f"      - ML Risk Score: {data['risk_score']:.1f}%")
    print(f"      ⚠️ Action: Immediate debt restructuring + fiscal consolidation\n")

print("\n2️⃣ COUNTRIES REQUIRING PREVENTIVE MEASURES:")
for country in moderate_stress:
    data = scorecard_data.loc[country]
    print(f"   🟡 {country}:")
    print(f"      - Fiscal Stress Score: {data['fiscal_stress_score']:.1f}/100")
    print(f"      - Primary concern: ", end="")
    if data['debt_to_gdp'] > 60:
        print("High debt burden")
    elif data['deficit_to_gdp'] < -5:
        print("Large fiscal deficit")
    elif data['revenue_to_gdp'] < 15:
        print("Weak revenue mobilization")
    else:
        print("Multiple moderate risks")
    print(f"      💡 Action: Enhanced monitoring + proactive policy adjustments\n")

print("\n3️⃣ UNIVERSAL POLICY PRIORITIES:")
print("   📈 Revenue Mobilization:")
print("      - Broaden tax base (reduce informality)")
print("      - Improve tax administration and compliance")
print("      - Rationalize tax incentives and exemptions")

print("\n   💰 Expenditure Rationalization:")
print("      - Prioritize growth-enhancing investments")
print("      - Reduce non-productive subsidies")
print("      - Improve public financial management")

print("\n   🌍 Debt Management:")
print("      - Extend debt maturity profiles")
print("      - Diversify financing sources")
print("      - Develop domestic capital markets")

print("\n   📊 Macroeconomic Stability:")
print("      - Control inflation through prudent monetary policy")
print("      - Manage exchange rate volatility")
print("      - Build fiscal buffers during good times")

print("\n" + "="*70)
print("✅ FISCAL SUSTAINABILITY ANALYSIS COMPLETE!")
print("="*70)

## ✅ Fiscal Sustainability Metrics Complete!

### Key Achievements:

1. **Comprehensive Debt Analysis**:
   - Debt-to-GDP ratios with sustainability thresholds
   - Temporal trends and cross-country comparisons
   - Debt service burden assessment

2. **Fiscal Balance Assessment**:
   - Budget deficit/surplus tracking
   - Revenue vs expenditure gaps
   - Structural vs cyclical deficits

3. **Revenue Efficiency Metrics**:
   - Revenue-to-GDP ratios
   - Tax effort indicators
   - Tax revenue composition

4. **Expenditure Composition**:
   - Total expenditure levels
   - Social spending allocation
   - Debt service crowding out

5. **Integrated Health Scorecard**:
   - Composite fiscal stress scores
   - Multi-dimensional sustainability assessment
   - Policy priority identification

6. **Sustainability Matrix**:
   - Debt-deficit positioning
   - Risk zone identification
   - Visual crisis warning system

### Outputs:
- `data/fiscal_metrics_complete.csv` - All fiscal indicators
- `data/fiscal_health_scorecard.csv` - Country stress scores
- `data/debt_sustainability_matrix.csv` - Debt-deficit matrix
- `data/focus_country_fiscal_metrics.csv` - Dashboard-ready data

### Next Steps:
1. **Social Spending Analysis** (`05_social_spending.ipynb`)
2. **Debt Projection Models** (`06_projections.ipynb`)
3. **Cross-Country Comparisons** (`07_peer_comparison.ipynb`)
4. **Interactive Dashboard** (Streamlit app)